In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob as gb
import cv2
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , Dense , Dropout , Flatten , MaxPooling2D , BatchNormalization ,experimental
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow import keras
from keras.models import Model

In [2]:
pwd

'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_6'

In [3]:
#data path
trainpath = 'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_6\\Orange_Dataset\\train'#
testpath = 'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_6\\Orange_Dataset\\test'

In [6]:
new_size=224
train_images=[]
train_labels=[]
for i in os.listdir(trainpath):#entering train folder
  print("Entering to the folder name:",i)
  files=gb.glob(pathname=str(trainpath+'/' + i + '/*.jpg'))# pointing to all the .jpg extension image folder
  print("Number of images in the folder is",len(files))
  for j in files:# reading each images
      class_cancer={'Anthracnose':0,'BlackSpot':1,'Healthy':2}
      image_raw=cv2.imread(j)
      image=cv2.cvtColor(image_raw,cv2.COLOR_BGR2RGB)
      resize_image=cv2.resize(image,(new_size,new_size))
      train_images.append(list(resize_image))
      train_labels.append(class_cancer[i])

Entering to the folder name: Anthracnose
Number of images in the folder is 30
Entering to the folder name: BlackSpot
Number of images in the folder is 30
Entering to the folder name: Healthy
Number of images in the folder is 30


In [7]:
new_size=224
test_images=[]
test_labels=[]
for i in os.listdir(testpath):# entering to the test folder
  print("Entering to the folder name:",i)
  files=gb.glob(pathname=str(testpath +'/' + i + '/*.jpg'))# pointing to all the .jpg extension image folder
  print("Number of images in the folder is",len(files))
  for j in files:
      class_cancer={'Anthracnose':0,'BlackSpot':1,'Healthy':2}
      image_raw=cv2.imread(j)
      image=cv2.cvtColor(image_raw,cv2.COLOR_BGR2RGB)
      resize_image=cv2.resize(image,(new_size,new_size))
      test_images.append(list(resize_image))
      test_labels.append(class_cancer[i])

Entering to the folder name: Anthracnose
Number of images in the folder is 24
Entering to the folder name: BlackSpot
Number of images in the folder is 25
Entering to the folder name: Healthy
Number of images in the folder is 25


In [8]:
def list_to_array_train(train_images,train_labels):
  return np.array(train_images),np.array(train_labels)

X_train,y_train=list_to_array_train(train_images,train_labels)


def list_to_array_test(test_images,test_labels):
  return np.array(test_images),np.array(test_labels)


X_test,y_test=list_to_array_test(test_images,test_labels)

In [9]:
print(X_train.shape)
print("*"*20)
print(y_train.shape)
print("*"*20)
print(X_test.shape)
print(y_test.shape)

(90, 224, 224, 3)
********************
(90,)
********************
(74, 224, 224, 3)
(74,)


In [10]:
def keras_to_categorical(y_train,y_test):
  return to_categorical(y_train),to_categorical(y_test)

y_train,y_test=keras_to_categorical(y_train,y_test)

In [11]:
def model_vgg16():
  VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224, 3))
  #Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
  for layer in VGG_model.layers:
    layer.trainable = False #True for actual transfer learning
  feature=keras.layers.GlobalAveragePooling2D()(VGG_model.output)
  d1=Dense(units=64,kernel_initializer="glorot_uniform", activation='relu')(feature)
  d2=Dense(units=32,kernel_initializer="glorot_uniform", activation='sigmoid')(d1)
  d3=Dense(units=3,kernel_initializer="glorot_uniform", activation='softmax')(d2)
  output = Model(inputs =VGG_model.input, outputs =d3)
  
  return output

model16=model_vgg16()

In [12]:
model16.compile(optimizer='Adam', loss='mse', metrics='accuracy')

In [13]:
accuracies = []

# Define the number of runs
num_runs = 3

for run in range(num_runs):
    # Fit the model
    history = model16.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=5, verbose=1, validation_data=(X_test, y_test))
    
    # Evaluate the model on the test set
    _, accuracy = model16.evaluate(X_test, y_test, verbose=0)
    
    # Append the accuracy to the list
    accuracies.append(accuracy)
    
    # Print the accuracy for each run
    print(f"Accuracy for run {run + 1}: {accuracy}")

# Calculate the mean accuracy
mean_accuracy = np.mean(accuracies)

# Print the mean accuracy
print(f"Mean accuracy across {num_runs} runs: {mean_accuracy}")

Epoch 1/5
18/18 [==============================] - 15s 756ms/step - loss: 0.1642 - accuracy: 0.7222 - val_loss: 0.0782 - val_accuracy: 1.0000
Epoch 2/5
18/18 [==============================] - 12s 700ms/step - loss: 0.0401 - accuracy: 1.0000 - val_loss: 0.0320 - val_accuracy: 1.0000
Epoch 3/5
18/18 [==============================] - 12s 704ms/step - loss: 0.0150 - accuracy: 1.0000 - val_loss: 0.0153 - val_accuracy: 1.0000
Epoch 4/5
18/18 [==============================] - 12s 704ms/step - loss: 0.0094 - accuracy: 1.0000 - val_loss: 0.0112 - val_accuracy: 1.0000
Epoch 5/5
18/18 [==============================] - 12s 706ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.0095 - val_accuracy: 1.0000


NameError: name 'history' is not defined

# Feature Extraction

In [14]:
def model_vgg16():
  VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224, 3))
  #Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
  for layer in VGG_model.layers:
    layer.trainable = False #True for actual transfer learning
  feature=keras.layers.GlobalAveragePooling2D()(VGG_model.output)
  
  output = Model(inputs =VGG_model.input, outputs =feature)
  
  return output

model_FE_16=model_vgg16()

In [15]:
model_FE_16.compile(optimizer='Adam', loss='mse', metrics='accuracy')
train_feature_16=model_FE_16.predict(X_train)
test_feature_16=model_FE_16.predict(X_test)

3/3 [==============================] - 5s 1s/step


In [17]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf=rf.fit(train_feature_16,y_train)
train_pred=rf.predict(train_feature_16)
test_pred=rf.predict(test_feature_16)
print("Train Accuracy Score",accuracy_score(train_pred,y_train))
print("Test Accuracy Score",accuracy_score(test_pred,y_test))

Train Accuracy Score 1.0
Test Accuracy Score 1.0


In [18]:
def model_vgg19():
  VGG_model = VGG19(weights='imagenet', include_top=False, input_shape=(224,224, 3))
  #Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
  for layer in VGG_model.layers:
    layer.trainable = False #True for actual transfer learning
  feature=keras.layers.GlobalAveragePooling2D()(VGG_model.output)
  ##d1=Dense(units=256,kernel_initializer="glorot_uniform", W_regularizer=l2(0.001), activation='relu')(feature)
  ##d2=Dense(units=2,kernel_initializer="glorot_uniform", W_regularizer=l2(0.001), activation='softmax')(d1)
  #output = Model(inputs =VGG_model.input, outputs =d2)
  output = Model(inputs =VGG_model.input, outputs =feature)
  
  return output

model_FE_19=model_vgg19()

In [19]:
model_FE_19.compile(optimizer='Adam', loss='mse', metrics='accuracy')
train_feature_19=model_FE_19.predict(X_train)
test_feature_19=model_FE_19.predict(X_test)

3/3 [==============================] - 7s 2s/step


In [20]:
final_train=np.hstack((train_feature_16,train_feature_19))
final_test=np.hstack((test_feature_16,test_feature_19))

In [21]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf=rf.fit(final_train,y_train)
train_pred=rf.predict(final_train)
test_pred=rf.predict(final_test)
print("Train Accuracy Score",accuracy_score(train_pred,y_train))
print("Test Accuracy Score",accuracy_score(test_pred,y_test))

Train Accuracy Score 1.0
Test Accuracy Score 1.0
